# Agentic RAG - Interactive Notebook

This notebook demonstrates the agentic RAG system by importing from `agentic_rag.py`.

**Architecture:**
```
agentic_rag.py    ← Single source of truth (all agent logic)
    ↑
app.py            ← Streamlit UI (imports AgenticRAG)
    ↑
agentic_rag.ipynb ← This notebook (imports AgenticRAG)
```

## Learning Objectives
1. Understand the **Reason → Retrieve → (Web Search) → Synthesize** workflow
2. Experiment with queries interactively
3. Inspect sources and reasoning steps
4. Add custom documents to the knowledge base

## 1. Setup

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install langchain langchain-community langchain-groq langgraph \
#     langchain_huggingface faiss-cpu tavily-python python-dotenv langchain-tavily

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# Optional: Enable LangSmith tracing
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
    os.environ["LANGCHAIN_PROJECT"] = "agentic-rag-notebook"
    print("✅ LangSmith tracing enabled")
else:
    print("⚪ LangSmith tracing disabled")

print(f"\nAPI Status:")
print(f"  Groq: {'✅' if GROQ_API_KEY else '❌'}")
print(f"  Tavily: {'✅' if TAVILY_API_KEY else '❌'}")

## 2. Initialize the Agent

Import `AgenticRAG` from our core module.

In [ ]:
from agentic_rag import AgenticRAG, SAMPLE_DOCUMENTS, AgentState

print("Initializing agent...")
agent = AgenticRAG(
    groq_api_key=GROQ_API_KEY,
    tavily_api_key=TAVILY_API_KEY,
)
print("✅ Agent ready!")

## 3. Explore the Knowledge Base

See what documents are pre-loaded.

In [ ]:
print(f"Knowledge base has {len(SAMPLE_DOCUMENTS)} documents:\n")
for i, doc in enumerate(SAMPLE_DOCUMENTS, 1):
    print(f"{i}. {doc['title']}")
    print(f"   {doc['content'][:100].strip()}...\n")

## 4. Test Retrieval

Query the vector store directly.

In [ ]:
test_query = "What is RAG?"
docs = agent.vector_store.similarity_search(test_query, k=2)

print(f"Query: {test_query}\n")
for i, doc in enumerate(docs, 1):
    print(f"Result {i}: {doc.metadata.get('title')}")
    print(f"{doc.page_content[:150]}...\n")

## 5. Run Full Agent Pipeline

Execute queries through the complete **Reason → Retrieve → Synthesize** loop.

In [ ]:
def ask(question: str):
    """Helper to run a query and display results nicely."""
    print(f"❓ {question}\n")
    print("🤔 Thinking...\n")
    
    result = agent.query(question)
    
    print("=" * 60)
    print("ANSWER:")
    print("=" * 60)
    print(result["answer"])
    
    print("\n" + "=" * 60)
    print("REASONING STEPS:")
    print("=" * 60)
    for step in result["reasoning_steps"]:
        print(f"  {step}")
    
    print("\n" + "=" * 60)
    print(f"SOURCES: {len(result['sources'])} used")
    print("=" * 60)
    for src in result["sources"]:
        print(f"  • {src['title']} ({src['type']})")
    
    return result

In [ ]:
# Test with a knowledge-base question
result = ask("What is RAG and what are its key benefits?")

In [ ]:
# Test with a question that triggers web search
result = ask("What are the latest developments in AI agents in 2024?")

In [ ]:
# Test comparing LangGraph and other frameworks
result = ask("How does LangGraph help build agentic RAG systems?")

## 6. Add Custom Documents

Extend the knowledge base with your own content.

In [ ]:
# Add a custom document
custom_docs = [
    """FlowNest is a productivity startup focused on AI-powered workflow automation.
    Key products include: intelligent task routing, automated document processing,
    and natural language workflow builders. Founded in 2023."""
]
custom_titles = ["About FlowNest"]

agent.add_documents(custom_docs, custom_titles)
print("✅ Added custom document")

In [ ]:
# Query about the custom document
result = ask("What is FlowNest and what products do they offer?")

## 7. Inspect the Agent Graph

Visualize the LangGraph workflow.

In [ ]:
# Try to visualize the graph
try:
    from IPython.display import Image, display
    display(Image(agent.graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization not available: {e}")
    print("\nGraph structure:")
    print("  START → reason → retrieve → [web_search | synthesize] → END")

## 8. Interactive Mode

Chat with the agent in a loop.

In [ ]:
# Interactive chat (run this cell and type queries)
print("Interactive mode. Type 'quit' to exit.\n")

while True:
    try:
        query = input("You: ").strip()
        if query.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        if not query:
            continue
        
        result = agent.query(query)
        print(f"\nAssistant: {result['answer']}")
        print(f"[{len(result['sources'])} sources]\n")
    except KeyboardInterrupt:
        print("\nGoodbye!")
        break

## Summary

This notebook demonstrates:

1. **Importing** the `AgenticRAG` class from `agentic_rag.py`
2. **Querying** the vector store directly
3. **Running** the full agent pipeline with web search fallback
4. **Adding** custom documents to extend the knowledge base
5. **Inspecting** reasoning steps and sources

To run the Streamlit app:
```bash
streamlit run app.py
```